# 全局解释器锁(GIL)

In [1]:
def Count(n):
    while(n):
        n-=1
%timeit Count(100000000)

7.03 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
# 多线程加速
from threading import Thread
n=100000000
t1=Thread(target=Count,args=(n//2,))
t2=Thread(target=Count,args=(n//2,))
import time
start=time.perf_counter()
t1.start()
t2.start()
t1.join()
t2.join()
end=time.perf_counter()
print('used:{}'.format(end-start))

used:7.094354336000038


通过上面的使用发现多线程并没有加速`Count`函数的计算，这是因为每个线程在Cpython解释器中执行时，会锁住当前线程，阻止别的线程执行，因此Python中的多线程是通过交错执行，模拟并行，这是为了解决两个问题：
- CPython采用引用计数管理内存(`sys.getrefcount()`)，当引用计数为0时，自动释放内存，因此为了规避多个线程同时引用一个实例，导致引用计数的race condition(竞争风险问题)，可能会出现引用计数只加1，导致内存污染(一个线程释放后，如果引用计数为0，实例内存回收，下一个线程无法引用该实例)。
- CPython使用了大量C语言库，这些库很多都不是原生线程安全的(线程安全降低性能和增加复杂度)。

线程在执行前获取GIL，执行完毕后，释放GIL。CPython还有一个轮询机制避免线程不释放GIL，时间间隔为15毫秒。

## 线程安全

因为GIL的原因，Python中的线程是线性执行的，但是由于轮询抢占机制，还是必须考虑线程安全(线程A获取x的值后后，CPU执行权被线程B拿走了后，修改了x的值，因此线程A的x值存在问题)。

In [74]:
n=0
def f():
    global n
    n+=1

threads=[]
# 如果是线性，那么结果理论上应该为100000，有时结果可能为99999
def test():
    for _ in range(100000):
        threads.append(Thread(target=f))

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

test()
print(n)

100000


In [75]:
import dis
dis.dis(f)

  4           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


这是因为`f`函数的字节码可能会被打断，为了保证线程安全，还是需要使用锁来保证线程安全。

In [76]:
n=0
import threading
threads=[]
lock=threading.Lock()

def f():
    global n
    with lock:
        n+=1

# 如果是线性，那么结果理论上应该为100000，有时结果可能为99999
def test():
    for _ in range(100000):
        threads.append(Thread(target=f))

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

test()
print(n)

100000


## 绕过GIL
- 绕过CPython，使用JPython等实现。
- 将关键性能代码使用高性能语言实现。
- 使用用C语言实现的库，如numpy。

## 问题
- 为什么CPU密集型的应用，多线程的性能比单线程差？
CPU密集型的应用中，多线程切换会消耗部分时间；相反，在IO密集型中，CPU消耗不大，通过执行其他的线程能有效提升CPU的利用率。
- GIL是一个好的设计吗？
Python3中改用计时器，循环执行线程，但是这样并没有解决GIL导致单一时间内只能执行一个线程。Python充分利用多核CPU，只能使用多进程，因为每个进程都有一个GIL，互不干扰。